# Script for crafting Adversarial Examples

## Perturbation Methods
#### 1. leet speak & unicode
#### 2. typos
#### 3. punctuation marks

In [1]:
import os
import time
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

import utils.text_processing as tp

In [2]:
# get data from important word detector
%store -r important_words_packages
%store -r sentence_packages
%store -r loo_results

%store -r important_words_packages_dev
%store -r sentence_packages_dev
%store -r loo_results_dev

In [3]:
#important_words_packages = important_words_packages_dev
#sentence_packages = sentence_packages_dev
#loo_results = loo_results_dev

## M3th0d 1: 133t 5p34k

#### 1. Create modified Dataset

In [4]:
# get original sentences from sentence_packages
original_sentences = []
for package in sentence_packages:
    original_sentences.append(package['original_sentence'])

In [5]:
# generate modified words: take Important Words and conduct modification

modified_words_packages = []

for important_words in important_words_packages:
    modified_words = []
    
    for word in important_words:
        modified_word_variances = []
        modified_word_variances.append(tp.to_leet(word))
        modified_words.append(modified_word_variances)

    modified_words_packages.append(modified_words)

In [6]:
# generate modified sentences
modifyable_original_sentences, modified_sentence_packages, number_of_modified_sentences = tp.generate_modified_sentence_packages(original_sentences, important_words_packages, modified_words_packages)

In [7]:
print('Total number of original sentences:', len(original_sentences))
print('Total number of modifyable original sentences: ', len(modifyable_original_sentences))
print('Total number of modified sentences: ', number_of_modified_sentences)

Total number of original sentences: 8
Total number of modifyable original sentences:  7
Total number of modified sentences:  11


#### 2. Import BERT Model

Get individual key from key file.

In [8]:
k = open("security/key.txt", "r")
key = k.read()

In [9]:
import torch
import random
import spacy
import numpy as np
import transformers
import pandas as pd

from absa import Predictor
from security import Authorization

nlp = spacy.load('en_core_web_sm')
pred = Predictor(os.path.join('models','en-laptops-absa'))

!export CUDA_VISIBLE_DEVICE=0

Initializing Predictor
Loading model models/en-laptops-absa
Config loaded from models/en-laptops-absa/config.json
Aspects loaded from models/en-laptops-absa/aspects.jsonl
Config loaded from models/en-laptops-absa/config.json


#### 3. Prediction

###### Original Prediction

In [9]:
documents = []
for sentence in modifyable_original_sentences:
    document = {'text': sentence, 'segments':[{'span':[0,0],'text': sentence}]}
    documents.append(document)
    
results = pred.predict(documents, key, with_segments=True)

original_predictions = []
for result in results:
    original_predictions.append(result[0])

Running authorization for token for functionality Analysis/Aspect-Sentiments and language None
DEBUG:security.authorization:Running authorization for token for functionality Analysis/Aspect-Sentiments and language None


### Modified Prediction

In [10]:
# 1. create indices for prediction,
# 2. flatten modified_sentence_packages and 
# 3. predict flattened list

results = []
documents = []
package_indices = []
package_index = 0
for sentence in modified_sentence_packages:
    package_index += 1
    for word in sentence:
        for variant in word:  
            package_indices.append(package_index)
            document = {'text': variant, 'segments':[{'span':[0,0],'text': variant}]}
            documents.append(document)
    
results = pred.predict(documents, key, with_segments=True)

modified_results_flattened = []
for result in results:
    modified_results_flattened.append(result[0])

Running authorization for token for functionality Analysis/Aspect-Sentiments and language None
DEBUG:security.authorization:Running authorization for token for functionality Analysis/Aspect-Sentiments and language None


In [11]:
# 4. map to lvl2 sentence packages
# IMPORTANT: here I loose the last variants level because I do not actually need it
# todo: loose levels --> only have one level per sentence when generating modified sentence packages

modified_predictions = []
modified_sentence = []
check = 1
for e, result in enumerate(modified_results_flattened):
    i = package_indices[e]    
    if i == check:
        modified_sentence.append(result)
    else:
        modified_predictions.append(modified_sentence)
        modified_sentence = []
        modified_sentence.append(result)
    check = i
modified_predictions.append(modified_sentence)


## 4. Comparison of results to check effectiveness of attack

In [12]:
leet_results = tp.compare_results(original_predictions, modified_predictions)

### 5. Creation of adversarial Dataset

In [13]:
original_texts, original_results, modified_texts, modified_results, successfull_modifications = tp.generate_results_lists(leet_results)

In [14]:
adversarial_dataset = pd.DataFrame(list(zip(original_texts, original_results, modified_texts, modified_results)),
                 columns = ['original_sentence', 'original_prediction', 'modified_sentence', 'modified_prediction'])

In [15]:
adversarial_dataset.to_json(r'data/adversarial_dataset_l33t.json', orient='records')

In [16]:
adversarial_dataset.head()

,original_sentence,original_prediction,modified_sentence,modified_prediction
0,super fast processor and really nice graphics ...,"[{'aspect': 'Graphics', 'sentiment': 'POS'}, {...",[super f45t processor and really nice graphics...,"[[{'aspect': 'Graphics', 'sentiment': 'POS'}, ..."
1,and plenty of storage with 250 gb(though I wil...,"[{'aspect': 'Storage', 'sentiment': 'POS'}]",[and plenty of 5t0r4g3 with 250 gb(though I wi...,[[]]
2,This computer is really fast and I'm shocked a...,"[{'aspect': 'Performance', 'sentiment': 'POS'}]",[This computer is really f45t and I'm shocked ...,"[[{'aspect': 'Laptop (general)', 'sentiment': ..."
3,i would really recommend to any person out the...,"[{'aspect': 'Laptop (general)', 'sentiment': '...",[i would really recommend to any person out th...,"[[{'aspect': 'Laptop (general)', 'sentiment': ..."
4,and its really cheap and you wont regret buyin...,"[{'aspect': 'Laptop (general)', 'sentiment': '...",[and its really ch34p and you wont regret buyi...,"[[{'aspect': 'Laptop (general)', 'sentiment': ..."


In [17]:
adversarial_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 790 entries, 0 to 789
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   original_sentence    790 non-null    object
 1   original_prediction  790 non-null    object
 2   modified_sentence    790 non-null    object
 3   modified_prediction  790 non-null    object
dtypes: object(4)
memory usage: 24.8+ KB


### 6. Results

In [18]:
results_table_leet = tp.generate_results_df(original_sentences, modifyable_original_sentences, number_of_modified_sentences, successfull_modifications, pmethod='leet')

In [19]:
%store results_table_leet
results_table_leet

Stored 'results_table_leet' (DataFrame)


,0
Perturbation Method,leet
Tokenizer,en_core_web_sm
Model,en-laptops-absa
Dataset,SemEval 2015 Laptops
Total number of original sentences,943
Total number of modifyable original sentences,897
Total number of modified sentences,2232
Total number of changed predictions through modification,1702
Success Rate,0.762545


In [ ]:
array = confusion_matrix(original_predictions, modified_predictions)
df_cm = pd.DataFrame(array, range(5), range(5))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 10}, fmt="d", linewidths=.1) # font size

plt.show()